In [ ]:
# Boilerplate: This block goes into every notebook.
# It sets up the environment, installs the requirements, and checks for the required environment variables.

import os
from IPython.display import clear_output
from dotenv import load_dotenv
import os


requirements_installed = False
max_retries = 3
retries = 0
REQUIRED_ENV_VARS = [
    "GROQ_API_KEY",
    "FIRECRAWL_API_KEY",
    "SERPER_API_KEY",
    "GEMINI_API_KEY",
]


def install_requirements():
    """Installs the requirements from requirements.txt file"""
    global requirements_installed
    if requirements_installed:
        print("Requirements already installed.")
        return

    print("Installing requirements...")
    install_status = os.system("pip install -r requirements.txt")
    if install_status == 0:
        print("Requirements installed successfully.")
        requirements_installed = True
    else:
        print("Failed to install requirements.")
        if retries < max_retries:
            print("Retrying...")
            retries += 1
            return install_requirements()
        exit(1)
    return


def setup_env():
    """Sets up the environment variables"""

    def check_env(env_var):
        value = os.getenv(env_var)
        if value is None:
            print(f"Please set the {env_var} environment variable.")
            exit(1)
        else:
            print(f"{env_var} is set.")

    load_dotenv()

    variables_to_check = REQUIRED_ENV_VARS

    for var in variables_to_check:
        check_env(var)


install_requirements()
setup_env()
clear_output()
print("🚀 Setup complete. Continue to the next cell.")

In [8]:
from uuid import uuid4
from typing import List, Dict
import google.generativeai as genai
import os
import traceback
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlparse, urljoin
import json
from markdownify import markdownify as md

gemini_api_key = os.getenv("GEMINI_API_KEY")
DEFAULT_GEMINI_MODEL = "gemini-1.5-flash"
DEFAULT_MAX_TOKENS = 1024
DEFAULT_TIMEOUT = 30  # seconds
DEFAULT_ZEN_CONTEXT_PATH = "data/zen/zen_context_v1.json"
DEFAULT_OPENAI_MODEL = "gpt-4o"


class NaiveWebCrawler:
    """A super simple implementation of a web crawler."""

    def get_links_from_page(self, url: str, timeout=DEFAULT_TIMEOUT) -> List[str]:
        """Extract the links from a page."""
        html = self.get_page_html(url, timeout=timeout)
        if not html:
            print(f"Failed to get HTML from {url}.")
            return []
        links = self.__extract_domain_links_from_html(url, html)
        if not links or len(links) == 0:
            print(f"No links found on {url}.")
        return links

    def get_page_html(self, url: str, timeout=DEFAULT_TIMEOUT) -> str:
        """Retrieves the HTML of a page."""
        try:
            response = requests.get(url, timeout=timeout)
            response.raise_for_status()
            return response.text
        except Exception as e:
            print(f"Failed to get HTML from {url}.")
            traceback.print_exc()
            return ""

    def get_page_markdown(self, url: str) -> str:
        """Retrive the contents of the page in markdown format."""
        try:
            html = self.get_page_html(url)
            markdown = md(html)
            return markdown
        except Exception as e:
            print(f"Failed to get HTML from {url}.")
            traceback.print_exc()
            return ""

    def __extract_domain_links_from_html(self, url: str, html: str) -> List[str]:
        """Extracts the links from the HTML."""
        try:
            soup = BeautifulSoup(html, "html.parser")
            # Extract the domain of the input URL
            parsed_url = urlparse(url=url)
            base_domain = parsed_url.netloc

            same_domain_urls = set()

            for anchor in soup.find_all("a", href=True):
                href = anchor["href"]
                full_url = urljoin(url, href)
                full_url_parsed = urlparse(full_url)
                if full_url_parsed.netloc == base_domain:
                    same_domain_urls.add(full_url)

            return list(same_domain_urls)
        except Exception as e:
            print(f"Failed to extract links from {url}.")
            traceback.print_exc()
            return []


class NaiveZenAgent:
    """A naive Zen Agent."""

    UM_WIKI_DEFINITIONS_PAGE = (
        "https://wiki.unifiedmindfulness.com/index.php/Category:Definitions"
    )

    def __init__(
        self,
        model=DEFAULT_GEMINI_MODEL,
        context_store="data/zen/zen_context_v1.json",
        expand_contract=None,
    ):
        """Initializes the Zen agent."""
        print("🧘🏻‍♂️ Initializing Zen Agent!")
        self.id = uuid4()
        self.model = model
        self.context_loaded = False
        self.crawler = NaiveWebCrawler()
        self.context_store = context_store
        self.__load_context()
        self.ai = self.__get_gemini_client(
            model=model
        )  # We'll default to Gemini for now because of the large context window.
        self.expand_contract = expand_contract
        print("🧘🏻‍♂️ Initialied Zen Agent!")

    def __get_gemini_client(self, model=DEFAULT_GEMINI_MODEL):
        """Builds and returns the Google Gemini client with the specified model."""
        genai.configure(api_key=gemini_api_key)
        model = genai.GenerativeModel(model_name=model)
        return model

    def __load_context(self) -> Dict[str, str]:
        """Loads the context for the agent from the UM Wiki and returns it."""
        try:
            with open(self.context_store, "r") as f:
                self.context = json.load(f)
                print(f"Loaded context from the context store at {self.context_store}.")
                self.context_loaded = True
            if self.context_loaded:
                return self.context
        except Exception as e:
            print(
                f"Failed to load context from the context store from {self.context_store}."
            )
            traceback.print_exc()
            self.context_loaded = False

        self.context = {}
        additional_links = [
            "https://wiki.unifiedmindfulness.com/index.php/Category:Terms_of_Service"  # Terms of Service
        ]
        links = self.crawler.get_links_from_page(url=self.UM_WIKI_DEFINITIONS_PAGE)
        links.extend(additional_links)
        print(f"Found {len(links)} links on the UM Wiki Definitions page.")
        for link in links:
            markdown = self.crawler.get_page_markdown(link)
            self.context[link] = markdown
        self.context_loaded = True
        return self.context

    def get_context(self):
        """Returns the context of the agent."""
        return self.context

    def __generate(self, prompt, max_tokens=DEFAULT_MAX_TOKENS):
        """Generates a response for a given prompt."""
        try:
            response = self.model.generate_content(query=prompt, max_tokens=max_tokens)
            return response
        except Exception as e:
            print(f"Failed to generate LLM response for {prompt}.")
            traceback.print_exc()

    def get_source(self):
        """Returns the Source (Expansion & Contraction)."""
        return self.expand_contract

In [9]:
from openai import OpenAI


class ExpandContract:
    def __init__(self, ai_model=DEFAULT_OPENAI_MODEL):
        """Initializes ExpansionContraction."""
        self.ai_model = ai_model
        self.ai = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
        self.system = (
            "You are 'Expand Contract', the primordial forces of the Universe."
        )

    def expand(self, sense_percept: str) -> str:
        """An expansion operation."""
        prompt = f"Expand: {sense_percept}"
        response = self.ai.chat.completions.create(
            messages=[
                {"role": "system", "content": self.system},
                {"role": "user", "content": prompt},
            ],
            model=self.ai_model,
        )
        return response.choices[0].message.content

    def contract(self, sense_percept: str) -> str:
        """A contraction operation."""
        prompt = f"Contract: {sense_percept}"
        response = self.ai.chat.completions.create(
            messages=[
                {"role": "system", "content": self.system},
                {"role": "user", "content": prompt},
            ],
            model=self.ai_model,
        )
        return response.choices[0].message.content

In [ ]:
zen_agent = NaiveZenAgent()

print("🚀 Setup complete. Continue to the next cell.")

zen_context = zen_agent.get_context()
zen_context_json = json.dumps(zen_context, indent=4)
print(zen_context_json)

In [10]:
expandcontract = ExpandContract()

sense_percept = (
    "Meditation and Mindfulness improves the quality of life for the average human."
)


expanded_text = expandcontract.expand(sense_percept=sense_percept)

print(f"Sense Percept: {sense_percept}")
print(f"Expansion: {expanded_text}")

Sense Percept: Meditation and Mindfulness improves the quality of life for the average human.
Expansion: Meditation and mindfulness are practices that have garnered increasing attention for their profound impact on the quality of life in a multitude of ways. At their core, these practices involve fostering a heightened state of awareness and presence, often leading to a harmonious balance between the mind and body.

To begin with, meditation serves as a powerful tool to reduce stress and anxiety. In a world where individuals are frequently inundated with constant stimuli and demands, taking the time to meditate allows for the cultivation of calmness and clarity. This, in turn, contributes to reduced levels of cortisol, the stress hormone, thereby promoting a healthier cardiovascular system and potentially extending lifespan.

Mindfulness, on the other hand, emphasizes living in the present moment and accepting it without judgment. This approach can lead to improved emotional regulation

In [ ]:
expandcontract = ExpandContract()

sense_percept = """
Meditation and mindfulness practices have garnered increasing attention and scientific validation for their profound effects on enhancing the quality of life for the average human being. Fundamentally, both meditation and mindfulness involve cultivating a heightened state of awareness and focused attention, allowing individuals to become more present in each moment.
Engaging regularly in meditation, a practice that often involves techniques such as deep breathing, visualization, and concentration, can lead to a myriad of psychological and physiological benefits. Research has shown that individuals who practice meditation experience reduced levels of stress, anxiety, and depression. This is largely attributed to the practice's ability to activate the parasympathetic nervous system, promoting relaxation and reducing the production of stress hormones such as cortisol.
Mindfulness, which can be thought of as a type of meditation in action, involves maintaining a moment-by-moment awareness of thoughts, feelings, bodily sensations, and the surrounding environment. By practicing mindfulness, individuals can decrease their automatic response to negative stimuli and develop a more compassionate and accepting attitude toward life’s challenges. This, in turn, leads to improved emotional regulation, making it easier to deal with adversity and maintain a balanced emotional state.
Moreover, mindfulness and meditation practices have been linked with improvements in cognitive function. Techniques associated with these practices can enhance attention span, increase mental clarity, and improve memory retention. As individuals become more adept at focusing their attention and cultivating an awareness of their mental processes, they often report greater productivity and creativity in their personal and professional lives.
Physical health benefits are also noteworthy. Regular practitioners of meditation and mindfulness report experiencing lower blood pressure, improved immune function, and better sleep quality. These physical health improvements contribute further to the overall enhancement of life quality, as they enable individuals to engage more actively and joyously in their daily activities.
Socially, mindfulness and meditation equip individuals with greater empathy and the ability to communicate more effectively. By being more present and attentive in their interactions, individuals can build deeper, more meaningful relationships, enriching their social experiences and community involvement.
Perhaps most importantly, mindfulness and meditation offer a path toward self-discovery and personal growth. By cultivating a non-judgmental awareness of their internal experiences, individuals often gain insights into their habitual patterns, enabling them to make conscious choices that align with their values and life aspirations.
In essence, the practices of meditation and mindfulness provide a comprehensive approach to improving life quality. By fostering psychological well-being, cognitive enhancement, physical health, and enriched social interactions, these practices empower individuals to live more fully, joyously, and with a greater sense of purpose.
"""

contracted_text = expandcontract.contract(sense_percept=sense_percept)

print(f"Sense Percept: {sense_percept}")
print(f"Contraction: {contracted_text}")

In [11]:
from langchain_openai import ChatOpenAI
from browser_use import Agent, controller
import asyncio
from pydantic import BaseModel

links = []

prompt = """
    - Go to YouTube and visit the 'expandcontract' channel of Shinzen Young. 
    - Get the YouTube video links of ALL the videos on the channel.
    - Don't miss out a single video!

"""


class WebpageInfo(BaseModel):
    link: str


@controller.action(
    "Check if YouTube video link is already extracted.", param_model=WebpageInfo
)
def go_to_webpage(webpage_info: WebpageInfo):
    return webpage_info.link


@controller.action(
    "Extract YouTube video links from the webpage.", param_model=WebpageInfo
)
def store_links(webpage_info: WebpageInfo):
    global links
    if not webpage_info.link in links:
        print(f"Storing link {webpage_info.link}")
        links.append(webpage_info.link)
    else:
        print(f"Link {webpage_info.link} already stored.")


async def main():
    agent = Agent(
        task="Find a one-way flight from Bali to Oman on 12 January 2025 on Google Flights. Return me the cheapest option.",
        llm=ChatOpenAI(model="gpt-4o"),
    )
    result = await agent.run()
    print(result)


asyncio.run(main())

ModuleNotFoundError: No module named 'browser_use'